In [ ]:
%pip install pandas natsort numpy

In [1]:
import pandas as pd
import json
from natsort import natsort_keygen

participant_usernames = [
    'ejthue',
    'amj5r6',
    'a2vium',
    'i2wsfv',
    'sq2mde',
    '4r7uba',
    'rnp6q7',
    'vf83re',
    '7g99vx',
    '4tcgxa',
    'kv3jvq',
    'sa896f',
    'mg53k5',
    'xxq7cw',
]
participant_ID_map = {username: f'P{i + 1}' for i, username in enumerate(participant_usernames)}

participant_time_adjustments = {
    'ejthue': {
        'natural-language-processing': {
            'quiz': -1 * 60
        }
    },
    'amj5r6': {
        'natural-language-processing': {
            'interaction': -1 * 60
        }
    },
    'rnp6q7': {
        'natural-language-processing': {
            'quiz': -3 * 60
        }
    },
    '4tcgxa': {
        'data-analysis': {
            'interaction': -2 * 60,
            'quiz': -3 * 60
        }
    },
    'kv3jvq': {
        'data-analysis': {
            'quiz': -1 * 60
        }
    }
}

# Process client data

In [2]:
with open('data/raw-client.json') as file:
    client_data = json.load(file)

participants_data = []
participant_tasks_data = []
participant_interaction_data = []

for participant in client_data:
    if participant['username'] not in participant_ID_map:
        continue

    participant_ID = participant_ID_map[participant['username']]
    
    # Participants (ID/group)
    participants_data.append({
        'participant_ID': participant_ID, 
        'group': participant['group']
    })

    # Participant tasks (time to complete interaction/quiz per taks)
    participant_tasks = {}

    for step in participant['steps']:
        if step['key'].startswith('interaction-'):
            type = 'interaction'
        elif step['key'].startswith('quiz-'):
            type = 'quiz'
        else:
            continue

        task = step['key'].split('-', 1)[1]
        if task not in participant_tasks:
            participant_tasks[task] = {}

        time = round((pd.to_datetime(step['endTime'], utc=True) - pd.to_datetime(step['startTime'], utc=True)).total_seconds())

        participant_tasks[task][type] = time

    for task, times in participant_tasks.items():
        interaction_time = times['interaction']
        quiz_time = times['quiz']

        # Apply time adjustments
        if participant['username'] in participant_time_adjustments and task in participant_time_adjustments[participant['username']]:
            if 'interaction' in participant_time_adjustments[participant['username']][task]:
                interaction_time += participant_time_adjustments[participant['username']][task]['interaction']
            if 'quiz' in participant_time_adjustments[participant['username']][task]:
                quiz_time += participant_time_adjustments[participant['username']][task]['quiz']

        participant_tasks_data.append({
            'participant_ID': participant_ID, 
            'task': task, 
            'interaction_time': interaction_time, 
            'quiz_time': quiz_time
        })

    # Participant interactions (input/response pairs with optional internal questions and mental state per task)
    messages = participant['interactionMessages']
    messages.sort(key=lambda x: x['order'])
    participant_interaction_turns = {}

    for message in messages:
        if message['task'] not in participant_interaction_turns:
            participant_interaction_turns[message['task']] = []

        if message['type'] == 'user':
            participant_interaction_turns[message['task']].append({
                'input': message['content'], 
                'response': None,
                'questions': None,
                'mental_state': None,
                'response_time': message['timeMS'] / 1000
            })
        elif message['type'] == 'ai':
            participant_interaction_turns[message['task']][-1]['response'] = message['content']
        elif message['type'] == 'internal':
            if participant_interaction_turns[message['task']][-1]['questions'] is None:
                participant_interaction_turns[message['task']][-1]['questions'] = message['content']
            elif participant_interaction_turns[message['task']][-1]['mental_state'] is None:
                participant_interaction_turns[message['task']][-1]['mental_state'] = message['content']

    for task, turns in participant_interaction_turns.items():
        for i, turn in enumerate(turns):
            participant_interaction_data.append({
                'participant_ID': participant_ID, 
                'task': task,
                'turn': i + 1, 
                **turn
            })

client_participants_df = pd.DataFrame(participants_data)

client_participant_tasks_df = pd.DataFrame(participant_tasks_data)
client_participant_tasks_df['interaction_time'] = client_participant_tasks_df['interaction_time'].astype('Int64')
client_participant_tasks_df['quiz_time'] = client_participant_tasks_df['quiz_time'].astype('Int64')

client_participant_interaction_df = pd.DataFrame(participant_interaction_data)

# Process Qualtrics data

## Pre-study survey

In [3]:
columns = {
    'user': 'username',
    'Q1': 'study_program',
    'Q2': 'study_stage',
    'Q3_1': 'experience_programming_estimated',
    'Q3_7': 'experience_oop',
    'Q3_2': 'experience_python',
    'Q3_3': 'experience_data_analysis',
    'Q3_5': 'experience_nlp',
    'Q4': 'familiarity_LLM',
    'Q5_1': 'familiarity_LLM_programming_writing',
    'Q5_2': 'familiarity_LLM_programming_modifying',
    'Q5_3': 'familiarity_LLM_programming_debugging',
    'Q5_4': 'familiarity_LLM_programming_explaining',
    'Q5_5': 'familiarity_LLM_programming_learning',
    'Q6': 'familiarity_LLM_programming_other',
    'Q7': 'LLM_programming_feedback',
}
survey_pre_df = pd.read_csv('data/raw-survey-pre.csv', skiprows=(1, 2), usecols=columns.keys()).rename(columns=columns)
survey_pre_df['participant_ID'] = survey_pre_df['username'].map(participant_ID_map)
survey_pre_df = survey_pre_df[~survey_pre_df['participant_ID'].isna()]

survey_pre_df['familiarity_LLM'] = survey_pre_df['familiarity_LLM'].map({
    'Never': 1,
    'Rarely': 2,
    'Sometimes': 3,
    'Often': 4,
    'Very often': 5,
})

for column in [
    'familiarity_LLM_programming_writing',
    'familiarity_LLM_programming_modifying',
    'familiarity_LLM_programming_debugging',
    'familiarity_LLM_programming_explaining',
    'familiarity_LLM_programming_learning',
]:
    survey_pre_df[column] = survey_pre_df[column].map({
        'Never': 1,
        'Rarely': 2,
        'Sometimes': 3,
        'Often': 4,
        'Almost always': 5,
    })

survey_pre_df['experience_programming'] = survey_pre_df[['experience_programming_estimated', 'experience_oop']].mean(axis=1)

survey_pre_df['study_programming_oriented'] = survey_pre_df['study_program'].map({
    'Not programming-oriented': False,
    'Programming-oriented (Computer Science, Artificial Intelligence, etc.)': True,
})
survey_pre_df['study_stage'] = survey_pre_df['study_stage'].map({
    'Master\'s, any year': 'M',
    'Bachelor, third year': 'B3',
    'Bachelor, second year': 'B2',
    'Bachelor, first year': 'B1',
})

survey_pre_df = survey_pre_df.sort_values('participant_ID', key=natsort_keygen()).reset_index(drop=True)

## Quizzes

In [4]:
def clean_quiz(task):
    quiz_df = pd.read_csv(f'data/raw-quiz-{task}.csv', skiprows=(1, 2)).rename(columns={'user': 'username'})
    quiz_df['participant_ID'] = quiz_df['username'].map(participant_ID_map)
    quiz_df = quiz_df[~quiz_df['participant_ID'].isna()]

    quiz_df = quiz_df[['participant_ID'] + [column for column in quiz_df.columns if column.startswith('Q')]]

    quiz_df['task'] = task

    return quiz_df

quiz_answers_df = pd.concat([clean_quiz('natural-language-processing'), clean_quiz('data-analysis')])

quiz_answers_df = quiz_answers_df.sort_values(['participant_ID', 'task']).reset_index(drop=True)
columns = quiz_answers_df.columns.tolist()
columns.sort()
columns.insert(0, columns.pop(columns.index('task')))
columns.insert(0, columns.pop(columns.index('participant_ID')))
quiz_answers_df = quiz_answers_df[columns]

## Chatbot evaluation

In [5]:
def clean_chatbot_evaluation(chatbot):
    columns = {
        'user': 'username',
        'Q1_1': 'usefulness_1',
        'Q1_2': 'usefulness_2',
        'Q1_3': 'usefulness_3',
        'Q1_4': 'usefulness_4',
        'Q1_5': 'usefulness_5',
        'Q1_6': 'usefulness_6',
        'Q1_7': 'ease_of_use_1',
        'Q1_8': 'ease_of_use_2',
        'Q1_9': 'ease_of_use_3',
        'Q1_10': 'ease_of_use_4',
        'Q1_11': 'ease_of_use_5',
        'Q1_12': 'ease_of_use_6',
        'Q2_1': 'cognitive_load_1',
        'Q2_2': 'cognitive_load_2',
        'Q2_3': 'cognitive_load_3',
        'Q2_4': 'cognitive_load_4',
        'Q2_5': 'cognitive_load_5',
        'Q2_6': 'cognitive_load_6',
        'Q3': 'feedback',
    }

    if chatbot == 'B':
        columns.update({
            'Q4': 'relative_speed',
            'Q5': 'speed_feedback'
        })

    evaluation_df = pd.read_csv(f'data/raw-evaluation-{chatbot}.csv', skiprows=(1, 2), usecols=columns.keys()).rename(columns=columns)
    evaluation_df['participant_ID'] = evaluation_df['username'].map(participant_ID_map)
    evaluation_df = evaluation_df[~evaluation_df['participant_ID'].isna()]

    for i in range(6):
        for column in [
            f'usefulness_{i + 1}',
            f'ease_of_use_{i + 1}',
        ]:
            evaluation_df[column] = evaluation_df[column].map({
                'Extremely disagree': 1,
                'Quite disagree': 2,
                'Slightly disagree': 3,
                'Neither agree nor disagree': 4,
                'Slightly agree': 5,
                'Quite agree': 6,
                'Extremely agree': 7,
            })

    if chatbot == 'B':
        evaluation_df['relative_speed'] = evaluation_df['relative_speed'].map({
            'Much slower': 1,
            'Somewhat slower': 2,
            'About the same': 3,
            'Somewhat faster': 4,
            'Much faster': 5,
        })

    evaluation_df['chatbot'] = chatbot

    return evaluation_df

chatbots_evaluation_df = pd.concat([clean_chatbot_evaluation('A'), clean_chatbot_evaluation('B')])

columns = chatbots_evaluation_df.columns.tolist()
columns.insert(0, columns.pop(columns.index('chatbot')))
columns.insert(0, columns.pop(columns.index('participant_ID')))
chatbots_evaluation_df = chatbots_evaluation_df[columns]

# Merge and save cleaned data 

In [6]:
participants_df = client_participants_df.merge(
    survey_pre_df[[
        'participant_ID', 
        'study_programming_oriented',
        'study_stage',
        'experience_programming',
        'experience_programming_estimated',
        'experience_oop',
        'experience_python',
        'experience_data_analysis',
        'experience_nlp',
        'familiarity_LLM',
        'familiarity_LLM_programming_writing',
        'familiarity_LLM_programming_modifying',
        'familiarity_LLM_programming_debugging',
        'familiarity_LLM_programming_explaining',
        'familiarity_LLM_programming_learning',
        'familiarity_LLM_programming_other',
        'LLM_programming_feedback',
    ]], 
    on='participant_ID', 
    how='outer'
)
participants_df.sort_values(['participant_ID'], key=natsort_keygen()).to_csv('data/participants.csv', index=False)


client_participant_interaction_df.sort_values(['participant_ID', 'task', 'turn'], key=natsort_keygen()).to_csv('data/interactions.csv', index=False)
 
tasks_df = client_participant_tasks_df.merge(
    quiz_answers_df, 
    on=['participant_ID', 'task'], 
    how='outer'
)
tasks_df.sort_values(['participant_ID', 'task'], key=natsort_keygen()).to_csv('data/tasks.csv', index=False)

chatbots_evaluation_df.sort_values(['participant_ID', 'chatbot'], key=natsort_keygen()).to_csv('data/evaluations.csv', index=False)

# Coding

In [7]:
import pandas as pd
import json

# Create DataFrames for design config
with open('../task/design.json') as file:
    design_config = json.load(file)

stage_tasks_df = pd.DataFrame([
    {'stage': stage, 'task': task}
    for stage, stage_config in design_config['stages'].items()
    for task in stage_config['tasks']
])
group_stage_models_df = pd.DataFrame([
    {
        'group': group,
        'stage': stage,
        'model': model,
    }
    for group, group_config in design_config['groups'].items()
    for stage, model in group_config['models'].items()
])
group_task_model_df = group_stage_models_df.merge(stage_tasks_df, on='stage')[['group', 'task', 'model']]

participants = pd.read_csv('data/participants.csv')
participants.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
})

interactions = pd.read_csv('data/interactions.csv')
interactions = interactions.merge(participants_df[['participant_ID', 'group']], on=['participant_ID'])
interactions = interactions.merge(group_task_model_df, on=['group', 'task'])
interactions.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
})

evaluations = pd.read_csv('data/evaluations.csv')
evaluations = evaluations.merge(participants[['participant_ID', 'group']], on='participant_ID')
evaluations['model'] = evaluations.apply(lambda row: design_config['groups'][row['group']]['models'][row['chatbot']], axis=1)
evaluations.style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Scan for accidental sensitive data submission

#### Input messages

In [8]:
scanned_participants = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14']
interactions[~interactions['participant_ID'].isin(scanned_participants)][['participant_ID', 'task', 'turn', 'input']]

,participant_ID,task,turn,input


#### Pre-study survey

In [9]:
scanned_participants = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14']
participants[~participants['participant_ID'].isin(scanned_participants)][['participant_ID', 'familiarity_LLM_programming_other', 'LLM_programming_feedback']]

,participant_ID,familiarity_LLM_programming_other,LLM_programming_feedback


#### Evaluations

In [10]:
scanned_participants = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14']
evaluations[~evaluations['participant_ID'].isin(scanned_participants)][['participant_ID', 'chatbot', 'feedback', 'speed_feedback']]

,participant_ID,chatbot,feedback,speed_feedback


### Export coding template

In [11]:
with pd.ExcelWriter('data/coding-template.xlsx') as writer:
    participants[['participant_ID', 'familiarity_LLM_programming_other']].to_excel(writer, sheet_name='LLM_programming_other', index=False)
    participants[['participant_ID', 'LLM_programming_feedback']].to_excel(writer, sheet_name='LLM_programming_feedback', index=False)
    evaluations[['participant_ID', 'model', 'chatbot', 'feedback']].to_excel(writer, sheet_name='feedback', index=False)

    speed_feedback_df = evaluations[evaluations['chatbot'] == 'B'][['participant_ID', 'model', 'speed_feedback']].copy()
    speed_feedback_df['compared_to'] = evaluations['model'].apply(lambda model: 'tom' if model == 'control' else 'control')
    speed_feedback_df[['participant_ID', 'model', 'compared_to', 'speed_feedback']].rename(columns={
        'model': 'model B (second)',
        'compared_to': 'compared to model A (first)'
    }).to_excel(writer, sheet_name='speed_feedback', index=False)

    interactions[['participant_ID', 'task', 'turn', 'input', 'response']].to_excel(writer, sheet_name='inputs', index=False)
